In [ ]:
!git clone https://github.com/Tianfang-Zhang/CAS-ViT.git
%cd CAS-ViT

Cloning into 'CAS-ViT'...
remote: Enumerating objects: 158, done.
remote: Counting objects: 100% (158/158), done.
remote: Compressing objects: 100% (101/101), done.
remote: Total 158 (delta 55), reused 150 (delta 50), pack-reused 0 (from 0)
Receiving objects: 100% (158/158), 5.23 MiB | 3.08 MiB/s, done.
Resolving deltas: 100% (55/55), done.
/content/CAS-ViT/classification/CAS-ViT


In [ ]:
!python --version

Python 3.10.12


In [ ]:
!cat README.md


In [ ]:
!pip install torch==1.11.0 torchvision==0.12.0
!pip install timm==0.5.4
!pip install mmcv-full==1.5.3 -f https://download.openmmlab.com/mmcv/dist/cu111/torch1.8.0/index.html
!pip install mmdet==2.24.0
!pip install mmsegmentation==0.24.0

In [ ]:
!pip install tensorboardX

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 2.6 MB/s eta 0:00:00


In [ ]:
%cd /content/CAS-ViT/classification

/content/CAS-ViT/classification


In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import shutil
from sklearn.model_selection import train_test_split

dataset_path = '/content/drive/My Drive/DATASET - REDES NEURAIS/dataset'
output_path = '/content/drive/My Drive/DATASET - REDES NEURAIS/dataset_split'

for split in ['train', 'val', 'test']:
    os.makedirs(os.path.join(output_path, split, 'coca'), exist_ok=True)
    os.makedirs(os.path.join(output_path, split, 'fanta'), exist_ok=True)

def split_and_copy_images(class_name):
    class_path = os.path.join(dataset_path, class_name)
    images = [img for img in os.listdir(class_path) if img.endswith(('.jpg', '.jpeg', '.png'))]

    train, temp = train_test_split(images, test_size=0.3, random_state=42)
    val, test = train_test_split(temp, test_size=0.5, random_state=42)

    def move_files(file_list, split):
        for file_name in file_list:
            src = os.path.join(class_path, file_name)
            dst = os.path.join(output_path, split, class_name, file_name)
            shutil.copy(src, dst)

    move_files(train, 'train')
    move_files(val, 'val')
    move_files(test, 'test')

split_and_copy_images('coca')
split_and_copy_images('fanta')

Sem adapter

In [ ]:
!python main.py \
  --input_size 384 --batch_size 64 --epochs 30 \
  --model rcvit_xs --data_path "/content/drive/My Drive/DATASET - REDES NEURAIS/dataset_split/train" \
  --lr 5e-5 --min_lr 5e-5 \
  --weight_decay 0.05 --output_dir "/content/drive/My Drive/DATASET - REDES NEURAIS/output-sem-adapter" \
  --finetune "model/cas-vit-xs.pth" --warmup_epochs 3 \
  --drop_path 0 --data_set image_folder \
  --eval_data_path "/content/drive/My Drive/DATASET - REDES NEURAIS/dataset_split/val" \
  --model_ema_eval True --model_ema True --use_amp True

Not using distributed mode
Namespace(batch_size=64, epochs=30, update_freq=2, model='rcvit_xs', drop_path=0.0, input_size=384, layer_scale_init_value=1e-06, model_ema=True, model_ema_decay=0.9995, model_ema_force_cpu=False, model_ema_eval=True, opt='adamw', opt_eps=1e-08, opt_betas=None, clip_grad=None, momentum=0.9, weight_decay=0.05, weight_decay_end=None, lr=5e-05, layer_decay=1.0, min_lr=5e-05, warmup_epochs=3, warmup_steps=-1, warmup_start_lr=0, color_jitter=0.4, aa='rand-m9-mstd0.5-inc1', smoothing=0.1, train_interpolation='bicubic', crop_pct=None, reprob=0.0, remode='pixel', recount=1, resplit=False, mixup=0.0, cutmix=0.0, cutmix_minmax=None, mixup_prob=0.0, mixup_switch_prob=0.5, mixup_mode='batch', data_path='/content/drive/My Drive/DATASET - REDES NEURAIS/dataset_split/train', eval_data_path='/content/drive/My Drive/DATASET - REDES NEURAIS/dataset_split/val', nb_classes=1000, imagenet_default_mean_and_std=True, data_set='image_folder', output_dir='/content/drive/My Drive/DATA

Com adapter

In [ ]:
!python main.py \
  --input_size 384 --batch_size 64 --epochs 30 \
  --model rcvit_xs --data_path "/content/drive/My Drive/DATASET - REDES NEURAIS/dataset_split/train" \
  --lr 5e-5 --min_lr 5e-5 \
  --weight_decay 0.05 --output_dir "/content/drive/My Drive/DATASET - REDES NEURAIS/output-com-adapter" \
  --finetune "model/cas-vit-xs.pth" --warmup_epochs 3 \
  --drop_path 0 --data_set image_folder \
  --eval_data_path "/content/drive/My Drive/DATASET - REDES NEURAIS/dataset_split/val" \
  --model_ema_eval True --model_ema True --use_amp True

Not using distributed mode
Namespace(batch_size=64, epochs=30, update_freq=2, model='rcvit_xs', drop_path=0.0, input_size=384, layer_scale_init_value=1e-06, model_ema=True, model_ema_decay=0.9995, model_ema_force_cpu=False, model_ema_eval=True, opt='adamw', opt_eps=1e-08, opt_betas=None, clip_grad=None, momentum=0.9, weight_decay=0.05, weight_decay_end=None, lr=5e-05, layer_decay=1.0, min_lr=5e-05, warmup_epochs=3, warmup_steps=-1, warmup_start_lr=0, color_jitter=0.4, aa='rand-m9-mstd0.5-inc1', smoothing=0.1, train_interpolation='bicubic', crop_pct=None, reprob=0.0, remode='pixel', recount=1, resplit=False, mixup=0.0, cutmix=0.0, cutmix_minmax=None, mixup_prob=0.0, mixup_switch_prob=0.5, mixup_mode='batch', data_path='/content/drive/My Drive/DATASET - REDES NEURAIS/dataset_split/train', eval_data_path='/content/drive/My Drive/DATASET - REDES NEURAIS/dataset_split/val', nb_classes=1000, imagenet_default_mean_and_std=True, data_set='image_folder', output_dir='/content/drive/My Drive/DATA